# Question to Answer: How much has Canada's Exports in Mineral Products Grown since 2021 and what are future projections?

## Set-up

In [1]:
# Here we will set up a virtual environment and install the required packages. 
import sys
import os
import subprocess

venv_dir = './venv'

if not os.path.exists(venv_dir):
    print(f"Creating virtual environment in {venv_dir}...")
    subprocess.check_call([sys.executable, "-m", "venv", venv_dir])
else:
    print(f"Virtual environment already exists at {venv_dir}.")

%pip install --upgrade pip
%pip install -r requirements.txt



Virtual environment already exists at ./venv.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Library Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from IPython.display import display # allows display of pandas df as a table

In [3]:
# Settings
plt.style.use('fivethirtyeight') # style of plots
pd.set_option('display.max_columns', 500) # sets max viewable cols to 500 in a df if the df is really wide

In [4]:
from pullData import load_data_from_api

conn = sqlite3.connect('canada_exports_2022_2024.db')
table_name = "canada_export_data"
load_data_from_api(table_name=table_name, conn=conn)

Fetched:  68  records for period:  202201  cmdCode:  27
Fetched:  31  records for period:  202201  cmdCode:  26
Fetched:  50  records for period:  202201  cmdCode:  25
Fetched:  67  records for period:  202202  cmdCode:  27
Fetched:  23  records for period:  202202  cmdCode:  26
Fetched:  46  records for period:  202202  cmdCode:  25
Fetched:  74  records for period:  202203  cmdCode:  27
Fetched:  30  records for period:  202203  cmdCode:  26
Fetched:  55  records for period:  202203  cmdCode:  25
Fetched:  70  records for period:  202204  cmdCode:  27
Fetched:  29  records for period:  202204  cmdCode:  26
Fetched:  52  records for period:  202204  cmdCode:  25
Fetched:  78  records for period:  202205  cmdCode:  27
Fetched:  31  records for period:  202205  cmdCode:  26
Fetched:  52  records for period:  202205  cmdCode:  25
Fetched:  69  records for period:  202206  cmdCode:  27
Fetched:  34  records for period:  202206  cmdCode:  26
Fetched:  62  records for period:  202206  cmdCo

In [5]:
query = 'SELECT * FROM canada_export_data'

df = pd.read_sql_query(query, conn)
df

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdCode,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qtyUnitAbbr,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,pk
0,C,M,20220101,2022,1,202201,124,CAN,Canada,X,Export,0,W00,World,0,W00,World,H6,HS,1,27,"Mineral fuels, mineral oils and products of th...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,1,0.0,0,None,1.303335e+10,1.303335e+10,4,0,1,202201_124_X_0_27
1,C,M,20220101,2022,1,202201,124,CAN,Canada,X,Export,24,AGO,Angola,0,W00,World,H6,HS,1,27,"Mineral fuels, mineral oils and products of th...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,1.370780e+02,1.370780e+02,0,0,1,202201_124_X_24_27
2,C,M,20220101,2022,1,202201,124,CAN,Canada,X,Export,32,ARG,Argentina,0,W00,World,H6,HS,1,27,"Mineral fuels, mineral oils and products of th...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,5.305880e+04,5.305880e+04,0,0,1,202201_124_X_32_27
3,C,M,20220101,2022,1,202201,124,CAN,Canada,X,Export,36,AUS,Australia,0,W00,World,H6,HS,1,27,"Mineral fuels, mineral oils and products of th...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,1.745862e+05,1.745862e+05,0,0,1,202201_124_X_36_27
4,C,M,20220101,2022,1,202201,124,CAN,Canada,X,Export,44,BHS,Bahamas,0,W00,World,H6,HS,1,27,"Mineral fuels, mineral oils and products of th...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,1.830350e+02,1.830350e+02,0,0,1,202201_124_X_44_27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5465,C,M,20241201,2024,12,202412,124,CAN,Canada,X,Export,792,TUR,Türkiye,0,W00,World,H6,HS,1,25,"Salt; sulphur; earths, stone; plastering mater...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,5.405065e+04,5.405065e+04,0,0,1,202412_124_X_792_25
5466,C,M,20241201,2024,12,202412,124,CAN,Canada,X,Export,796,TCA,Turks and Caicos Isds,0,W00,World,H6,HS,1,25,"Salt; sulphur; earths, stone; plastering mater...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,3.416785e+03,3.416785e+03,0,0,1,202412_124_X_796_25
5467,C,M,20241201,2024,12,202412,124,CAN,Canada,X,Export,818,EGY,Egypt,0,W00,World,H6,HS,1,25,"Salt; sulphur; earths, stone; plastering mater...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,6.489351e+03,6.489351e+03,0,0,1,202412_124_X_818_25
5468,C,M,20241201,2024,12,202412,124,CAN,Canada,X,Export,826,GBR,United Kingdom,0,W00,World,H6,HS,1,25,"Salt; sulphur; earths, stone; plastering mater...",2,0,C00,TOTAL CPC,0,0,TOTAL MOT,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,None,6.805496e+04,6.805496e+04,0,0,1,202412_124_X_826_25


In [6]:
# data cleaning
# we do not need cols like flowCode, classificationSearchCode, isOriginalClassification, isLeaf, customsCode, customsDesc, mosCode, motCode, motDesc, isReported, isAggregate, cifvalue
clean_df = df.drop(['flowCode', 'flowDesc', 'partner2Code', 'partner2ISO', 'partner2Desc', 'classificationSearchCode', 'isOriginalClassification', 'isLeaf', 'customsCode', 'customsDesc', 'mosCode', 'motCode', 'motDesc', 'isReported', 'isAggregate', 'cifvalue', 'legacyEstimationFlag'], axis=1)
clean_df

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,partnerCode,partnerISO,partnerDesc,classificationCode,cmdCode,cmdDesc,aggrLevel,qtyUnitCode,qtyUnitAbbr,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,fobvalue,primaryValue,pk
0,C,M,20220101,2022,1,202201,124,CAN,Canada,0,W00,World,H6,27,"Mineral fuels, mineral oils and products of th...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,1,0.0,0,1.303335e+10,1.303335e+10,202201_124_X_0_27
1,C,M,20220101,2022,1,202201,124,CAN,Canada,24,AGO,Angola,H6,27,"Mineral fuels, mineral oils and products of th...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,1.370780e+02,1.370780e+02,202201_124_X_24_27
2,C,M,20220101,2022,1,202201,124,CAN,Canada,32,ARG,Argentina,H6,27,"Mineral fuels, mineral oils and products of th...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,5.305880e+04,5.305880e+04,202201_124_X_32_27
3,C,M,20220101,2022,1,202201,124,CAN,Canada,36,AUS,Australia,H6,27,"Mineral fuels, mineral oils and products of th...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,1.745862e+05,1.745862e+05,202201_124_X_36_27
4,C,M,20220101,2022,1,202201,124,CAN,Canada,44,BHS,Bahamas,H6,27,"Mineral fuels, mineral oils and products of th...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,1.830350e+02,1.830350e+02,202201_124_X_44_27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5465,C,M,20241201,2024,12,202412,124,CAN,Canada,792,TUR,Türkiye,H6,25,"Salt; sulphur; earths, stone; plastering mater...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,5.405065e+04,5.405065e+04,202412_124_X_792_25
5466,C,M,20241201,2024,12,202412,124,CAN,Canada,796,TCA,Turks and Caicos Isds,H6,25,"Salt; sulphur; earths, stone; plastering mater...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,3.416785e+03,3.416785e+03,202412_124_X_796_25
5467,C,M,20241201,2024,12,202412,124,CAN,Canada,818,EGY,Egypt,H6,25,"Salt; sulphur; earths, stone; plastering mater...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,6.489351e+03,6.489351e+03,202412_124_X_818_25
5468,C,M,20241201,2024,12,202412,124,CAN,Canada,826,GBR,United Kingdom,H6,25,"Salt; sulphur; earths, stone; plastering mater...",2,-1,N/A,0.0,0,-1,N/A,0.0,0,0.0,0,0.0,0,6.805496e+04,6.805496e+04,202412_124_X_826_25


In [ ]:
# let's do some data cleaning and tranformation
# we will convert the date column to datetime for clean plotting
transform_df = clean_df.copy()
transform_df['date'] = pd.to_datetime(clean_df['period'], format='%Y%m')
transform_df

ValueError: time data "202201" doesn't match format "%Y%m%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [8]:
# let's do some plotting
# let's plot the exports by month and year and see the trend

